In [1]:
# importing data using kaggle API
import kaggle 
!kaggle datasets download -d ankitbansal06/retail-orders

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
retail-orders.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# import neccessary libraries
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
import zipfile
zip_ref = zipfile.ZipFile('retail-orders.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
df = pd.read_csv("orders.csv")

In [5]:
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order Id          9994 non-null   int64 
 1   Order Date        9994 non-null   object
 2   Ship Mode         9993 non-null   object
 3   Segment           9994 non-null   object
 4   Country           9994 non-null   object
 5   City              9994 non-null   object
 6   State             9994 non-null   object
 7   Postal Code       9994 non-null   int64 
 8   Region            9994 non-null   object
 9   Category          9994 non-null   object
 10  Sub Category      9994 non-null   object
 11  Product Id        9994 non-null   object
 12  cost price        9994 non-null   int64 
 13  List Price        9994 non-null   int64 
 14  Quantity          9994 non-null   int64 
 15  Discount Percent  9994 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 1.2+ MB


In [7]:
df.shape

(9994, 16)

### Exploratory Data Analysis

In [8]:
df.isnull().any()

Order Id            False
Order Date          False
Ship Mode            True
Segment             False
Country             False
City                False
State               False
Postal Code         False
Region              False
Category            False
Sub Category        False
Product Id          False
cost price          False
List Price          False
Quantity            False
Discount Percent    False
dtype: bool

In [9]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', 'Not Available', 'unknown',
       'First Class', nan, 'Same Day'], dtype=object)

In [10]:
df = pd.read_csv('orders.csv' , na_values=['Not Available' , 'unknown'])

In [11]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [12]:
df.columns

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

In [13]:
# Rename columns make them lower case and adjust space 
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ' , '_')
df.head()

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [14]:
# Derive new columns : discount , sale_price , profit 
df['discount'] = ((df['list_price'] * df['discount_percent'] )/ 100.0).round(2)
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']

In [15]:
df['order_date']  = pd.to_datetime(df['order_date'] , format='%Y-%m-%d')

In [16]:
# drop unncessary columns 
df.drop(['country' , 'discount_percent' , 'cost_price' , 'list_price'] ,axis=1 , inplace=True)

In [17]:
df.duplicated().any()

False

In [20]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'city', 'state',
       'postal_code', 'region', 'category', 'sub_category', 'product_id',
       'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [22]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

load_dotenv()

username = os.getenv('MYSQL_USER')
encoded_password = quote_plus(os.getenv('MYSQL_PASSWORD'))
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
database_name = os.getenv('MYSQL_DB_NAME')

engine = create_engine(f"mysql+mysqlconnector://{username}:{encoded_password}@{host}:{port}/{database_name}")
df.to_sql("retail_orders", con=engine, if_exists="append", index=False)

9994